# Step 1 Python Dependence

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from itertools import product
from gridWalk import bicycleGridRiding
from DPforGridBike_v1 import ValueIteration

Python Settings

In [2]:
# 设置 numpy 科学计数法
np.set_printoptions(suppress=True, threshold=5000)
# 设置 pandas 的格式
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.float_format', lambda x: '%.8f' % x)
# Full Tensor
torch.set_printoptions(sci_mode=False, profile="full")
torch.autograd.set_detect_anomaly(True)

### 设置随机数种子

In [3]:
import random

# Step 2 Functions

In [4]:
class TrajProccess():
    def __init__(self, env, trajData, device):
        self.trajData = trajData
        self.env = env
        self.device = device

    @ property
    def maxTrajLength(self):
        '''
        Max Length For Given Trajectories
        :param trajData: Dataframe, Given Trajectories
        '''
        trajIDs = np.unique(self.trajData['TrajID'])

        trajectoryList = []
        trajectoryLength = []
        for trajID in trajIDs:
            trajectory = []
            trajID_Data = self.trajData[self.trajData['TrajID'] == trajID].copy()  # The Information of Current Trajectory

            # [state, action, nextState]
            trajID_Data.apply(lambda x: trajectory.append([x['state'], x['action'], x['nextState']]), axis=1)

            trajectoryList.append(trajectory)  # Current Trajectory
            trajectoryLength.append(len(trajectory))

        return trajectoryList, max(trajectoryLength)

    @property
    def Padding(self):
        '''
        Trajectories Padding Function
        :param trajectories:
        :param maxTL: Max Length Of Given Trajectories
        :return: trajMatrix, size = [numTraj, maxTL]
        '''
        trajectories = self.maxTrajLength[0]
        maxLength = self.maxTrajLength[1]

        trajectoryMatrix = []
        for trajectory in trajectories:
            if len(trajectory) != maxLength:
                for i in range((maxLength - len(trajectory))):
                    trajectory.append([-1, -1, -1])
                trajectoryMatrix.append(trajectory)
            else:
                trajectoryMatrix.append(trajectory)

        return torch.tensor(trajectoryMatrix, dtype=torch.float32).to(self.device)

    def find_svf(self):
        # print('Finding SVF ···')
        numStates = len(self.env.states)    # numStates
        # trajectoryList = self.Padding()    # []
        trajectoryList = self.maxTrajLength[0]    # []
        svf = np.zeros(numStates)  # [numStates, 1]

        for trajectory in trajectoryList:
            # print('轨迹为{}'.format(trajectory))
            for stateActionPair in trajectory:
                # [1373.0, 0.0, 1373.0]
                state = int(stateActionPair[0])
                svf[state] += 1
                # if state != -1:
                #     svf[state] += 1

        # svf /= trajectories.shape[0]
        # [[s,a], [s,a],[s,a]],
        # [[s,a], [s,a],[s,a]]
        svf /= len(trajectoryList)

        a = torch.tensor(svf, dtype=torch.float32).to(self.device)
        a1 = F.normalize(a, dim=0)
        
        return a1

    def find_expected_svf(self, r=None):
        # print('Finding Expected SVF ······')
        env = self.env

        # trajMatrix.shape -> [21237, 120, 3]
        trajMatrix = self.Padding    # [numTraj, maxLength, 3]
        n_trajectories = trajMatrix.shape[0]  # [numTrajectories, 1]
        trajectory_length = trajMatrix.shape[1]  # [trajectoryLength, 1]

        # ########### 修改了此处传参的过程 ###########
        # if r != None:
        #     agent.rewardFunc = r.cpu().detach().numpy()    # [numStates, 1]
        # else:
        #     agent.rewardFunc = [env.rewardFunc(agentloc) for agentloc in range(len(env.states))]    # 此处传参的过程可能存在问题
            
        ######## 5月11日修改结果  ######
        if r != None:
            r1 = r.cpu().detach().numpy()    # [numStates, 1]
        else:
            r1 = [env.rewardFunc(agentloc) for agentloc in range(len(env.states))]    # 此处传参的过程可能存在问题

        agent = ValueIteration(env, r1)
        GridV = agent.plan(threshold=0.01)
        policy = agent.get_policy(GridV)
        # Reward Function Is Not Available - Why ?    -> 没有将奖励函数传入至 Value Iteration 之中

        start_state_count = torch.zeros(len(env.states), dtype=torch.float32)  # [numStates, 1]
        for trajectory in trajMatrix:
            start_state_count[int(trajectory[0, 0])] += 1
        p_start_state = start_state_count / n_trajectories  # the Probability of Start State

        expected_svf = torch.tile(p_start_state, (trajectory_length, 1)).T    # [trajLength, numStates] -> [numStates, trajLength]
        for t in range(1, trajectory_length):
            expected_svf[:, t] = 0  # Trajectory 长度为 trajLength, 初始化为起始点出现的概率
            for i, j in product(range(len(env.states)), range(env.action_space.n)):
                action = env.index2Action(j)
                transitionProb = env.transitFunc(i, action)    # dict

                # Next State
                for k in transitionProb.keys():
                    expected_svf[k, t] += (expected_svf[i, t - 1] *
                                           policy[i, j] *  # Stochastic policy
                                           transitionProb[k])
            print('第{}个位置已经完成'.format(t))
            
        b = expected_svf.sum(dim=1, dtype=torch.float32).to(self.device)  # [numStates, 1]
        b1 = F.normalize(b, dim=0)
        
        return b1

图卷积得到的 Reward 传入矩阵之中，不存在道路的 reward 强制修改为某个特定的常数，以得到智能体环境的高效表示，避免噪音的影响；
- 这种情况下卷积神经网络能否得到相似的结果？
- 首先尝试卷积神经网络作为依赖学习的方式
- 似乎不需要将图卷积神经网络加入到函数的设计之中

更新日志
- 2024.5.18 9：05 按照上述的思路，在不加入卷积神经网络或者是图神经网络的基础上，学习其奖励以明确这种方式是否可以收敛 

In [5]:
class deepIRL():
    def __init__(self, env, stateFeatures, trajProcess,
                 ini_net, discount, device, epochs, passList,
                 learning_rate=0.1, l1=0.1, l2=0.1):
        self.env = env
        self.stateFeatures = stateFeatures
        self.passList = passList
        self.trajProcess = trajProcess
        self.ini_net = ini_net
        self.discount = discount
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l1 = l1
        self.l2 = l2
        self.device = device

    def update(self, net, alpha):
        # Forward Propagation
        # print('Forward Propagation ····· ')
        n_states, d_states = self.stateFeatures.shape  # numStates, dimension
        reward = torch.matmul(net(self.stateFeatures), alpha).reshape(n_states,).to(self.device)    # [numStates, dimensions]
        reward = (reward - reward.mean()) / reward.std()    # Reward Standardization

        # 2024.5.18 日晚更新：路径规划器仍然需要获得所有的奖励
        # 加入一个变量为 reward_temp 保存暂时的奖励
        reward_temp = torch.zeros(len(self.env.states))
        # passList 应该为一个可以通过的 FID 列表
        # rewardTemp 应该是一个与环境大小相同的网格单元
        i = 0
        for j in self.passList:
            reward_temp[j] = reward[i]
            i += 1
        
        # print(reward)
        weights = []
        biases = []
        for name, param in net.named_parameters():
            if 'weight' in name:
                weights.append(param)
            elif 'bias' in name:
                biases.append(param)

        # print('Back Propagation ······')
        adagrad_epsilon = 1e-6  # AdaGrad numerical stability.
        expected_svf = trajProcess.find_expected_svf(r=reward_temp).reshape(len(self.env.states),)  # torch.tensor, size=[numStates, 1] -> After Normalization
        svf = trajProcess.find_svf().reshape(len(self.env.states),)  # torch.tensor, size=[numStates, 1] -> After Normalization
        grad = svf - expected_svf  # [numStates, 1]
        grad1 = torch.Tensor([grad[s] for s in self.passList]).to(self.device)    # n_states

        updates = []
        hist_alpha_grad = torch.zeros(alpha.shape).to(self.device)  # [dimensions, 1]
        output = net(self.stateFeatures)    # [numStates, output_dim] -> [numStates, dimension] -> [numStates, 4]
        alpha_grad = (torch.matmul(grad1.T, output).
                      reshape(alpha.shape))  # [output_dim, 1] -> [dimensions, 1] -> [4, 1]
        hist_alpha_grad += alpha_grad ** 2  # history grad    [output_dim, 1] -> [dimensions, 1]
        adj_alpha_grad = alpha_grad / (adagrad_epsilon + torch.sqrt(hist_alpha_grad))
        updates.append((alpha, alpha + adj_alpha_grad * self.learning_rate))


        def grad_for_state(s, theta, svf_diff, r):
            """
            Calculate the gradient with respect to theta for one state.
            """
            regularisation = torch.sum(torch.abs(theta)) * self.l1 + torch.sum(theta ** 2) * self.l2
            autograd = torch.autograd.grad(r[s], theta, retain_graph=True)[0]
            svf_diff_s = svf_diff[s]
            return  svf_diff_s * autograd - regularisation
            

        hist_w_grads = [torch.zeros_like(weight).to(self.device) for weight in weights]
        for i, W in enumerate(weights):
            w_gradList = []
            for state in range(n_states):
                w_template_grad = grad_for_state(state, W, grad1, reward)
                w_gradList.append(w_template_grad)
            
            # 计算梯度
            w_grads = torch.stack(w_gradList)
            w_grad = torch.sum(w_grads, dim=0)
            # 清零历史梯度
            if hist_w_grads[i] is not None:
                hist_w_grads[i].zero_()
            # 更新历史梯度平方累积和
            hist_w_grads[i] += w_grad ** 2
            # 计算调整后的梯度
            adj_w_grad = w_grad / (adagrad_epsilon + torch.sqrt(hist_w_grads[i]))
            # 添加参数更新规则到更新列表
            updates.append((W, W + adj_w_grad * self.learning_rate))

        hist_b_grads = [torch.zeros_like(bias).to(self.device) for bias in biases]
        for i, b in enumerate(biases):
            # 计算梯度
            b_gradList = []
            for state in range(n_states):
                b_gradList.append(grad_for_state(state, b, grad1, reward))
            # 计算梯度
            b_grads = torch.stack(b_gradList)
            b_grad = torch.sum(b_grads, dim=0)
            # 清零历史梯度
            if hist_b_grads[i] is not None:
                hist_b_grads[i].zero_()
            # 更新历史梯度平方累积和
            hist_b_grads[i] += b_grad ** 2
            # 计算调整后的梯度
            adj_b_grad = b_grad / (adagrad_epsilon + torch.sqrt(hist_b_grads[i]))
            # 添加参数更新规则到更新列表
            updates.append((b, b + adj_b_grad * self.learning_rate))

        n = int((len(updates) - 1) / 2)
        with torch.no_grad():
            for name, param in net[0].named_parameters():
                if 'weight' in name:
                    param.data = updates[1][1]
                elif 'bias' in name:
                    param.data = updates[n+1][1]
            
            for name, param in net[1].named_parameters():
                if 'weight' in name:
                    param.data = updates[2][1]
                elif 'bias' in name:
                    param.data = updates[n+2][1]
                    
        net.zero_grad()
        rewardNew = torch.matmul(net(self.stateFeatures), updates[0][1]).reshape(n_states,)

        # 2024.5.19 日更新：对 reward 进行标准化操作
        rewardNew1 = (rewardNew.max()-rewardNew)/(rewardNew.max()-rewardNew.min())
        
        # ### 2024.5.18 9:25 此处更新，将所有不存在路段的网格奖励归 0
        # rewardNew1 = torch.zeros_like(rewardNew)
        # for loc in self.PassList:
        #     rewardNew1[loc] = rewardNew[loc]
            
        return updates[0][1], net, rewardNew1

    def train(self, n):
        ini_net = self.ini_net
        ############
        ini_alpha = torch.normal(mean=0, std=0.01, size=(4, 1)).to(self.device)    # 此处的代码也需要适应神经网络的变化
        # rewardList= []
        # svfDiff = []
        rewardList = []
        stdList = []
        num_epochs = self.epochs
        for i in range(n):
            with tqdm(total=int(num_epochs / n), desc='Iteration %d' % i) as pbar:
                for i_epoch in range(int(num_epochs / n)):
                    # 奖励的更新
                    alpha, output_net, reward = self.update(net=ini_net, alpha=ini_alpha)
                    ini_net = nn.Sequential(*list(output_net.children()))
                    ini_alpha = alpha
                    # rewardList.append(reward)
                    # svfDiff.append(svf)
                    rewardList.append(reward.mean())
                    stdList.append(reward.std())

                    if (i_epoch + 1) % n == 0:
                        pbar.set_postfix({
                            'epoch':
                                '%d' % (num_epochs / n * i + i_epoch + 1),
                            # 'return':
                            # '%.3f' % np.mean(return_list[-10:])
                        })
                    pbar.update(1)
                    
        return rewardList, stdList, reward, ini_alpha

# Step 3 创建智能体运行的环境并训练模型

In [6]:
print('The Script Is Running······')

The Script Is Running······


In [7]:
attrTable = pd.read_csv(r'./03_stateAttrNorm_output.txt', sep=',')
attrTable1 = pd.read_csv(r'./NormAttrPass.csv', sep=',')

In [8]:
os.chdir(r'F:\BaiduNetdiskDownload\共享单车轨迹\共享单车轨迹\01_研究生毕业论文\1_2_DataPreProcess')
trajFile = r'SAPairCom.txt'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trajData = pd.read_csv(trajFile)  # ['FID', 'OrderID', 'TrajID', 'timeStamp', 'LG_ID', 'state', 'action', 'nextState']

# Create Environment For Bicycle To Move
env = bicycleGridRiding(attrTable=attrTable)
state, _ = env.reset()
print('Environment Is Successfully Built')

trajProcess = TrajProccess(env=env, trajData=trajData, device=device)

Environment Is Successfully Built


In [9]:
passListIndex = attrTable1['FID'].values

In [10]:
# Inverse Reinforcement Learning
# Training
stateFeatures = torch.tensor(np.vstack([attrTable1[attrTable1['FID']==s].to_numpy()[0, 2:] for s in passListIndex]),
                                 dtype=torch.float32).to(device)
n_states, d_states = stateFeatures.shape    # numStates, dimension

In [24]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)

In [25]:
setup_seed(2024)

In [26]:
# Initialize Model Parameters
net = nn.Sequential(nn.Linear(23, 12),
                    nn.Linear(12, 6),
                    nn.BatchNorm1d(6),
                    nn.Softmax(),
                    nn.Linear(6, 4),
                    nn.BatchNorm1d(4),
                    nn.Softmax())
def init_weights(m):
    '''
    :param m: Neural Network Module
    :return:
    '''
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

net.apply(init_weights)
net.to(device=device)

Sequential(
  (0): Linear(in_features=23, out_features=12, bias=True)
  (1): Linear(in_features=12, out_features=6, bias=True)
  (2): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Softmax(dim=None)
  (4): Linear(in_features=6, out_features=4, bias=True)
  (5): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Softmax(dim=None)
)

In [27]:
n_states, d_states
ini_alpha = torch.normal(mean=0, std=0.01, size=(4, 1)).to(device)

In [28]:
ini_net = nn.Sequential(*list(net.children()))

agent = deepIRL(env=env, 
                stateFeatures=stateFeatures, 
                trajProcess=trajProcess, 
                ini_net=ini_net, 
                discount=0.9, 
                device=device, 
                epochs=18,
                passList=passListIndex)
rewardList, stdList, reward, ini_alpha = agent.train(n=18)

Iteration 0:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

第1个位置已经完成
第2个位置已经完成
第3个位置已经完成
第4个位置已经完成
第5个位置已经完成
第6个位置已经完成
第7个位置已经完成
第8个位置已经完成
第9个位置已经完成
第10个位置已经完成
第11个位置已经完成
第12个位置已经完成
第13个位置已经完成
第14个位置已经完成
第15个位置已经完成
第16个位置已经完成
第17个位置已经完成
第18个位置已经完成
第19个位置已经完成
第20个位置已经完成
第21个位置已经完成
第22个位置已经完成
第23个位置已经完成
第24个位置已经完成
第25个位置已经完成
第26个位置已经完成
第27个位置已经完成
第28个位置已经完成
第29个位置已经完成
第30个位置已经完成
第31个位置已经完成
第32个位置已经完成
第33个位置已经完成
第34个位置已经完成
第35个位置已经完成
第36个位置已经完成
第37个位置已经完成
第38个位置已经完成
第39个位置已经完成
第40个位置已经完成
第41个位置已经完成
第42个位置已经完成
第43个位置已经完成
第44个位置已经完成
第45个位置已经完成
第46个位置已经完成
第47个位置已经完成
第48个位置已经完成
第49个位置已经完成
第50个位置已经完成
第51个位置已经完成
第52个位置已经完成
第53个位置已经完成
第54个位置已经完成
第55个位置已经完成
第56个位置已经完成
第57个位置已经完成
第58个位置已经完成
第59个位置已经完成
第60个位置已经完成
第61个位置已经完成
第62个位置已经完成
第63个位置已经完成
第64个位置已经完成
第65个位置已经完成
第66个位置已经完成
第67个位置已经完成
第68个位置已经完成
第69个位置已经完成
第70个位置已经完成
第71个位置已经完成
第72个位置已经完成
第73个位置已经完成
第74个位置已经完成
第75个位置已经完成
第76个位置已经完成
第77个位置已经完成
第78个位置已经完成
第79个位置已经完成
第80个位置已经完成
第81个位置已经完成
第82个位置已经完成
第83个位置已经完成
第84个位置已经完成
第85个位置已经完成
第86个位置已经完成
第87个位置已经完成
第88个位置已经完成
第89个位置已经完成
第90个位置已经完成
第91个位置已经完成
第92个位置已经

Iteration 1:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

第1个位置已经完成
第2个位置已经完成
第3个位置已经完成
第4个位置已经完成
第5个位置已经完成
第6个位置已经完成
第7个位置已经完成
第8个位置已经完成
第9个位置已经完成
第10个位置已经完成
第11个位置已经完成
第12个位置已经完成
第13个位置已经完成
第14个位置已经完成
第15个位置已经完成
第16个位置已经完成
第17个位置已经完成
第18个位置已经完成
第19个位置已经完成
第20个位置已经完成
第21个位置已经完成
第22个位置已经完成
第23个位置已经完成
第24个位置已经完成
第25个位置已经完成
第26个位置已经完成
第27个位置已经完成
第28个位置已经完成
第29个位置已经完成
第30个位置已经完成
第31个位置已经完成
第32个位置已经完成
第33个位置已经完成
第34个位置已经完成
第35个位置已经完成
第36个位置已经完成
第37个位置已经完成
第38个位置已经完成
第39个位置已经完成
第40个位置已经完成
第41个位置已经完成
第42个位置已经完成
第43个位置已经完成
第44个位置已经完成
第45个位置已经完成
第46个位置已经完成
第47个位置已经完成
第48个位置已经完成
第49个位置已经完成
第50个位置已经完成
第51个位置已经完成
第52个位置已经完成
第53个位置已经完成
第54个位置已经完成
第55个位置已经完成
第56个位置已经完成
第57个位置已经完成
第58个位置已经完成
第59个位置已经完成
第60个位置已经完成
第61个位置已经完成
第62个位置已经完成
第63个位置已经完成
第64个位置已经完成
第65个位置已经完成
第66个位置已经完成
第67个位置已经完成
第68个位置已经完成
第69个位置已经完成
第70个位置已经完成
第71个位置已经完成
第72个位置已经完成
第73个位置已经完成
第74个位置已经完成
第75个位置已经完成
第76个位置已经完成
第77个位置已经完成
第78个位置已经完成
第79个位置已经完成
第80个位置已经完成
第81个位置已经完成
第82个位置已经完成
第83个位置已经完成
第84个位置已经完成
第85个位置已经完成
第86个位置已经完成
第87个位置已经完成
第88个位置已经完成
第89个位置已经完成
第90个位置已经完成
第91个位置已经完成
第92个位置已经

Iteration 2:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

...

Iteration 16:   0%|                                                                              | 0/1 [00:00<?, ?it/s]

第1个位置已经完成
第2个位置已经完成
第3个位置已经完成
第4个位置已经完成
第5个位置已经完成
第6个位置已经完成
第7个位置已经完成
第8个位置已经完成
第9个位置已经完成
第10个位置已经完成
第11个位置已经完成
第12个位置已经完成
第13个位置已经完成
第14个位置已经完成
第15个位置已经完成
第16个位置已经完成
第17个位置已经完成
第18个位置已经完成
第19个位置已经完成
第20个位置已经完成
第21个位置已经完成
第22个位置已经完成
第23个位置已经完成
第24个位置已经完成
第25个位置已经完成
第26个位置已经完成
第27个位置已经完成
第28个位置已经完成
第29个位置已经完成
第30个位置已经完成
第31个位置已经完成
第32个位置已经完成
第33个位置已经完成
第34个位置已经完成
第35个位置已经完成
第36个位置已经完成
第37个位置已经完成
第38个位置已经完成
第39个位置已经完成
第40个位置已经完成
第41个位置已经完成
第42个位置已经完成
第43个位置已经完成
第44个位置已经完成
第45个位置已经完成
第46个位置已经完成
第47个位置已经完成
第48个位置已经完成
第49个位置已经完成
第50个位置已经完成
第51个位置已经完成
第52个位置已经完成
第53个位置已经完成
第54个位置已经完成
第55个位置已经完成
第56个位置已经完成
第57个位置已经完成
第58个位置已经完成
第59个位置已经完成
第60个位置已经完成
第61个位置已经完成
第62个位置已经完成
第63个位置已经完成
第64个位置已经完成
第65个位置已经完成
第66个位置已经完成
第67个位置已经完成
第68个位置已经完成
第69个位置已经完成
第70个位置已经完成
第71个位置已经完成
第72个位置已经完成
第73个位置已经完成
第74个位置已经完成
第75个位置已经完成
第76个位置已经完成
第77个位置已经完成
第78个位置已经完成
第79个位置已经完成
第80个位置已经完成
第81个位置已经完成
第82个位置已经完成
第83个位置已经完成
第84个位置已经完成
第85个位置已经完成
第86个位置已经完成
第87个位置已经完成
第88个位置已经完成
第89个位置已经完成
第90个位置已经完成
第91个位置已经完成
第92个位置已经

Iteration 17:   0%|                                                                              | 0/1 [00:00<?, ?it/s]

第1个位置已经完成
第2个位置已经完成
第3个位置已经完成
第4个位置已经完成
第5个位置已经完成
第6个位置已经完成
第7个位置已经完成
第8个位置已经完成
第9个位置已经完成
第10个位置已经完成
第11个位置已经完成
第12个位置已经完成
第13个位置已经完成
第14个位置已经完成
第15个位置已经完成
第16个位置已经完成
第17个位置已经完成
第18个位置已经完成
第19个位置已经完成
第20个位置已经完成
第21个位置已经完成
第22个位置已经完成
第23个位置已经完成
第24个位置已经完成
第25个位置已经完成
第26个位置已经完成
第27个位置已经完成
第28个位置已经完成
第29个位置已经完成
第30个位置已经完成
第31个位置已经完成
第32个位置已经完成
第33个位置已经完成
第34个位置已经完成
第35个位置已经完成
第36个位置已经完成
第37个位置已经完成
第38个位置已经完成
第39个位置已经完成
第40个位置已经完成
第41个位置已经完成
第42个位置已经完成
第43个位置已经完成
第44个位置已经完成
第45个位置已经完成
第46个位置已经完成
第47个位置已经完成
第48个位置已经完成
第49个位置已经完成
第50个位置已经完成
第51个位置已经完成
第52个位置已经完成
第53个位置已经完成
第54个位置已经完成
第55个位置已经完成
第56个位置已经完成
第57个位置已经完成
第58个位置已经完成
第59个位置已经完成
第60个位置已经完成
第61个位置已经完成
第62个位置已经完成
第63个位置已经完成
第64个位置已经完成
第65个位置已经完成
第66个位置已经完成
第67个位置已经完成
第68个位置已经完成
第69个位置已经完成
第70个位置已经完成
第71个位置已经完成
第72个位置已经完成
第73个位置已经完成
第74个位置已经完成
第75个位置已经完成
第76个位置已经完成
第77个位置已经完成
第78个位置已经完成
第79个位置已经完成
第80个位置已经完成
第81个位置已经完成
第82个位置已经完成
第83个位置已经完成
第84个位置已经完成
第85个位置已经完成
第86个位置已经完成
第87个位置已经完成
第88个位置已经完成
第89个位置已经完成
第90个位置已经完成
第91个位置已经完成
第92个位置已经

Iteration 17: 100%|████████████████████████████████████████████████████████████████████| 1/1 [33:32<00:00, 2012.68s/it]


In [29]:
rewardList

[tensor(0.3515, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5325, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5238, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5150, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5149, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5124, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.5044, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4906, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4733, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4554, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4396, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4272, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4182, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4123, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4086, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4067, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.4059, device='cuda:0', grad_fn=<MeanBackward0>

In [30]:
stdList

[tensor(0.2599, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.2349, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.2188, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.2101, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1991, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1875, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1759, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1649, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1550, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1466, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1399, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1349, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1313, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1290, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1275, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1266, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1263, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(0.1262

In [31]:
reward

tensor([0.2184, 0.2602, 0.2411, 0.3117, 0.3832, 0.1816, 0.2425, 0.2251, 0.2679,
        0.1622, 0.2833, 0.1850, 0.2307, 0.3243, 0.2702, 0.3226, 0.2663, 0.3119,
        0.5509, 0.2577, 0.2686, 0.3330, 0.5841, 0.5999, 0.2736, 0.2638, 0.2578,
        0.3596, 0.3581, 0.5143, 0.4075, 0.4002, 0.2485, 0.3811, 0.3719, 0.4650,
        0.4254, 0.2513, 0.4670, 0.4231, 0.3910, 0.3673, 0.3820, 0.4022, 0.3308,
        0.2884, 0.3938, 0.3832, 0.4230, 0.4642, 0.3753, 0.4172, 0.4208, 0.3743,
        0.3788, 0.3897, 0.4060, 0.4571, 0.5126, 0.4414, 0.5021, 0.5410, 0.3675,
        0.3135, 0.3966, 0.4748, 0.4626, 0.4223, 0.4852, 0.4428, 0.4876, 0.4761,
        0.4134, 0.5172, 0.3599, 0.4224, 0.2997, 0.2744, 0.4057, 0.4535, 0.4598,
        0.3723, 0.5077, 0.4593, 0.4467, 0.3796, 0.3725, 0.3880, 0.2456, 0.3963,
        0.2081, 0.2022, 0.2645, 0.2686, 0.2833, 0.3044, 0.4256, 0.4787, 0.4561,
        0.3917, 0.4932, 0.5277, 0.2941, 0.4339, 0.3147, 0.3219, 0.2765, 0.3492,
        0.3645, 0.3756, 0.3898, 0.4353, 

保存运算所得的结果以及其参数

In [32]:
ini_alpha1 = ini_alpha.cpu().detach().numpy()
np.save('Alpha_pass_Final', ini_alpha1)

In [33]:
torch.save(net, './NetParam_Pass_Final.pkl')

In [34]:
reward_cpu = reward.cpu().detach().numpy()
dataFrame = pd.DataFrame({'index':passListIndex, 'reward':reward_cpu})
dataFrame.to_csv(r'./reward_Pass_Final.csv', encoding='utf-8')

In [38]:
# reward_List_cpu = rewardList.cpu().detach().numpy()
# stdList_cpu = stdList.cpu().detach().numpy()
df1 = pd.DataFrame({'index':[i for i in range(len(rewardList))], 
                    'reward':[rewardMean.cpu().detach().numpy() for rewardMean in rewardList], 
                    'std': [stdMean.cpu().detach().numpy() for stdMean in stdList]})

In [39]:
df1

,index,reward,std
0,0,0.35150898,0.25989157
1,1,0.532497,0.2349417
2,2,0.523759,0.21882361
3,3,0.5150271,0.2100878
4,4,0.5148948,0.19907239
5,5,0.5123568,0.18754125
6,6,0.50439245,0.1759208
7,7,0.49064386,0.16486835
8,8,0.47327292,0.15495202
9,9,0.45542097,0.14655997


In [40]:
df1.to_csv(r'./rewardTrend.csv', encoding='utf-8')

In [48]:
! pip install -r requirements0.txt

ERROR: Invalid requirement: 'Package                           Version' (from line 1 of requirements0.txt)


In [45]:
! pip freeze

aiobotocore @ file:///C:/b/abs_3cwz1w13nn/croot/aiobotocore_1701291550158/work
aiohttp @ file:///C:/b/abs_27h_1rpxgd/croot/aiohttp_1707342354614/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
altair @ file:///C:/b/abs_27reu1igbg/croot/altair_1687526066495/work
anaconda-anon-usage @ file:///C:/b/abs_95v3x0wy8p/croot/anaconda-anon-usage_1697038984188/work
anaconda-catalogs @ file:///C:/b/abs_8btyy0o8s8/croot/anaconda-catalogs_1685727315626/work
anaconda-client @ file:///C:/b/abs_34txutm0ue/croot/anaconda-client_1708640705294/work
anaconda-cloud-auth @ file:///C:/b/abs_410afndtyf/croot/anaconda-cloud-auth_1697462767853/work
anaconda-navigator @ file:///C:/b/abs_cfvv8k_j21/croot/anaconda-navigator_1704813334508/work
anaconda-project @ file:///C:/ci_311/anaconda-project_1676458365912/work
anyio @ file:///C:/b/abs_847uobe7ea/c

In [46]:
! pip freeze > requirements.txt

In [47]:
! pip install -r requirements.txt

Processing c:\b\abs_3cwz1w13nn\croot\aiobotocore_1701291550158\work (from -r requirements.txt (line 1))


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\b\\abs_3cwz1w13nn\\croot\\aiobotocore_1701291550158\\work'



In [49]:
! pip install pipreqs

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/798.3 kB ? eta -:--:--
   --- ------------------------------------ 61.4/798.3 kB 1.6 MB/s eta 0:00:01
   --- ------------------------------------ 61.4/798.3 kB 1.6 MB/s eta 0:00:01
   ---- ---------------------------------- 81.9/798.3 kB 762.6 kB/s eta 0:00:01
   ---- ---------------------------------- 92.2/798.3 kB 581.0 kB/s eta 0:00:02
   ---- ---------------------------------- 92.2/798.3 kB 581.0 kB/s eta 0:00:02
   ---- ---------------------------------- 92.2/798.3 kB 581.0 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/798.3 kB 385.0 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/798.3 kB 385.0 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/798.3 kB 385.0 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/798.3 kB 385.0 kB/s eta 0:00:02
   ------- ------------------

In [52]:
! pipreqs . --encoding=utf8

INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in .\requirements.txt


In [53]:
! pip install -r requirements.txt

     ---------------------------------------- 0.0/51.5 kB ? eta -:--:--
     ------- -------------------------------- 10.2/51.5 kB ? eta -:--:--
     ---------------------- --------------- 30.7/51.5 kB 325.1 kB/s eta 0:00:01
     ---------------------- --------------- 30.7/51.5 kB 325.1 kB/s eta 0:00:01
     ------------------------------ ------- 41.0/51.5 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 51.5/51.5 kB 219.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 1.4 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.1 MB 919.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 919.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.1 MB 476.3 kB/s eta 0:00:03
   ----- ---------------------------------- 0.1/1.1 MB 610.6 kB/s eta 0:00:02
   -------- ------------------------------- 0.2/1.1 MB 811.5 kB/s eta 0:00:02


  You can safely remove it manually.
